In [1]:
%matplotlib inline
import h5py
import numpy as np
import os
from UG2.utils import data as data_utils
from UG2.utils import image as image_utils
from UG2.config import Config
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import scipy.ndimage as im
import time
from UG2 import main

In [2]:
data_path            = "/data/UG2_data/h5Files/"
traindedModels_path  = "/data/UG2_data/trainedModels/"

In [3]:
## create blurred-dataset 
kernel_blurring_flag= True
num_files           = 10
factor              = 1
patch_size          = np.array([224, 224])
epochsList          = [50]
learningRateList    = [0.01]
numberTrainingKernel= [1]
flags               = [0,1]
numberOfExperiments = len(epochsList)*len(learningRateList)
dataset_name        = "imagenet"
blurSeed            = "RandomSeed"

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
trainStartIndex = 0
numberCompletedExperiments = 0
fileStartIndex  = 1
for i in range(trainStartIndex, len(epochsList)):
    for nLR in learningRateList:
        for nTK in numberTrainingKernel:
            config = Config() 
            config.epochs = epochsList[i]
            config.cuda = True
            config.data_parallel = False
            config.lr = nLR
            config.data_path = data_path
            config.data_files = []
            config.lr_step_list = [10,30]
            config.step_decay_param = 0.25
            for fileIndex in range(fileStartIndex,num_files+1):
                if kernel_blurring_flag:
                    
                    config.data_files.append(os.path.join(data_path, dataset_name+"_nTK_"+str(nTK)+ "_flag_" + "_".join([str(f) for f in flags])+"factor"+str(factor)+"nFile"+str(fileIndex)+blurSeed+".h5"))
                else:        
                    config.data_files.append(os.path.join(data_path, dataset_name+".h5"))
                    
            config.batch_size = 30
            config.print_step = 1
            config.checkpoint = 1
            config.exit_loss_window = 5
            config.loss_epsilon = 0.01
            config.resume_training_flag = False
            config.kernel_blurring_flag = False
            config.discriminator = "feat_ext"
            config.factor      = factor
            config.img_size    = 64
            config.num_workers = 8
            config.data_format = "h5"
            
            if i:
                config.epochs = epochsList[i]-epochsList[i-1]
                config.resume_training_flag = True
                config.resume_model_path = traindedModels_path
                config.resume_model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i-1])+".pth"

            config.model_path = traindedModels_path
            config.model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i])+"nTK"+str(nTK)+"nFile"+str(num_files)+blurSeed+".pth"
            numberCompletedExperiments = numberCompletedExperiments+1
            main.train(config)
        
            print("Completed training on file: "+str(num_files))        